Code for extracting super-voxel ROIS from the Keller/Zlatic data

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import copy
from pathlib import Path

import pandas as pd
import pyspark

from janelia_core.fileio.exp_reader import read_img_file
from keller_zlatic_vnc.raw_data_processing import video_to_supervoxel_baselines

## Parameters go here

In [3]:
ps = dict()

# Location of excel file specifying where the data for each experiment is saved relative to the base folder
ps['data_loc_file'] = r'A:\projects\keller_vnc\data\experiment_data_locations.xlsx'

# Parameters for where the brain mask is located
ps['brain_mask_file'] = r'A:\\projects\\keller_vnc\\data\\registered_brain_mask\thresholdedVoxelMapSmooth.klb'

# Name of the subfolder we create under the [Main Folder]\[Subfolder]\ directory to save extracted results
ps['extracted_folder'] = 'extracted\\rois_5_25_25'

# Additional parameters (constant for all datasets) specifying where the data is stored
ps['image_base_folder'] =r'K:\\SV4'
ps['image_processed_folder'] = 'Results\\WeightFused'
ps['image_ext'] = r'weightFused.TimeRegistration.templateSpace.klb'

# Parameters for how we preprocess each image
ps['0_sub_value'] = 100.0

# Parameters for how we extract supervoxels
ps['roi_extract_opts'] = {'voxel_size_per_dim':[5, 25, 25], 'brain_mask_perc': .75}

# Parameters for calculating baseline
ps['baseline_calc_opts'] = {'window_length': 41, 'filter_start': -20, 'write_offset': 20, 'p': .1, 'n_processes': 80}

## Read in excel file specifying location of each dataset

In [4]:
def c_fcn(str):
    return str.replace("'", "")
converters = {0:c_fcn, 1:c_fcn}

data_locs = pd.read_excel(ps['data_loc_file'], header=1, usecols=[1, 2], converters=converters)

## Load the brain mask

In [5]:
brain_mask_file = Path(ps['brain_mask_file'])
brain_mask = read_img_file(brain_mask_file)

## Create a spark Context

In [6]:
conf = pyspark.SparkConf().setMaster('local[20]').setAll([
    ('spark.executor.memory', '10g'), ('spark.driver.memory','400g'), ('spark.driver.maxResultSize', '300g')])
sc = pyspark.SparkContext(conf=conf)

## Process each dataset

In [7]:
def preprocess_f(x):
    x[x==0] = ps['0_sub_value'] 
    return x

In [8]:
roi_extract_opts = copy.copy(ps['roi_extract_opts'])
roi_extract_opts['brain_mask'] = brain_mask
roi_extract_opts['preprocess_f'] = preprocess_f

In [ ]:
n_subjects = len(data_locs)
for d_i in range(n_subjects):
    print('**********************************************************************************************')
    print('Processing subject ' + str(d_i + 1) + ' of ' + str(n_subjects))
    
    base_data_dir = Path(ps['image_base_folder']) / data_locs['Main folder'][d_i] / data_locs['Subfolder'][d_i] / ps['image_processed_folder']
    save_dir =  Path(ps['image_base_folder']) / data_locs['Main folder'][d_i] / data_locs['Subfolder'][d_i] / Path(ps['extracted_folder'])
    
    roi_info = video_to_supervoxel_baselines(base_data_dir=base_data_dir, save_dir=save_dir, 
                                             roi_extract_opts=roi_extract_opts, extract_params=ps,
                                             baseline_calc_opts=ps['baseline_calc_opts'], new_comp=False, sc=sc)

**********************************************************************************************
Processing subject 1 of 66
Save directory already exists: K:\SV4\CW_17-08-23\L1-561nm-ROIMonitoring_20170823_145226.corrected\extracted\rois_5_25_25
Beginning supervoxel extraction.
ROIs have already been extracted.  Using existing ROI information saved in: 
K:\SV4\CW_17-08-23\L1-561nm-ROIMonitoring_20170823_145226.corrected\extracted\rois_5_25_25\extracted_f.h5
K:\SV4\CW_17-08-23\L1-561nm-ROIMonitoring_20170823_145226.corrected\extracted\rois_5_25_25\roi_locs.pkl
Beginning baseline calculation.
Baselines have already been calculated.  Using baselines saved in: 
K:\SV4\CW_17-08-23\L1-561nm-ROIMonitoring_20170823_145226.corrected\extracted\rois_5_25_25\baseline_f.h5
**********************************************************************************************
Processing subject 2 of 66
Save directory already exists: K:\SV4\CW_17-08-23\L2-561nm-ROIMonitoring_20170823_161308.corrected\extracted\

Found 7336 images.
Extracting: 2826 super voxels from 7336 images.
Extracted 2826 ROIS in 787.3535466194153 seconds.
Beginning baseline calculation.
Baselines calculated in 34.284602642059326 seconds.
**********************************************************************************************
Processing subject 10 of 66
Save directory does not already exist.  Creating: K:\SV4\CW_17-08-26\L6-561nm-ROIMonitoring_20170826_183354.corrected\extracted\rois_5_25_25
Beginning supervoxel extraction.
Searching for image files...
Found 6702 images.
Extracting: 2826 super voxels from 6702 images.
Extracted 2826 ROIS in 708.541111946106 seconds.
Beginning baseline calculation.
Baselines calculated in 32.54045605659485 seconds.
**********************************************************************************************
Processing subject 11 of 66
Save directory does not already exist.  Creating: K:\SV4\CW_17-08-27\L1-561nm-ROIMonitoring_20170827_112137.corrected\extracted\rois_5_25_25
Beginning 

Baselines calculated in 30.904778480529785 seconds.
**********************************************************************************************
Processing subject 20 of 66
Save directory does not already exist.  Creating: K:\SV4\CW_17-09-01\L2-561nm-ROImonitoring_20170901_122419.corrected\extracted\rois_5_25_25
Beginning supervoxel extraction.
Searching for image files...
Found 6312 images.
Extracting: 2826 super voxels from 6312 images.
Extracted 2826 ROIS in 560.8550536632538 seconds.
Beginning baseline calculation.
Baselines calculated in 32.10803818702698 seconds.
**********************************************************************************************
Processing subject 21 of 66
Save directory does not already exist.  Creating: K:\SV4\CW_17-09-01\L3-561nm-ROImonitoring_20170901_162644.corrected\extracted\rois_5_25_25
Beginning supervoxel extraction.
Searching for image files...
Found 6555 images.
Extracting: 2826 super voxels from 6555 images.
Extracted 2826 ROIS in 637.54

Found 6324 images.
Extracting: 2826 super voxels from 6324 images.
Extracted 2826 ROIS in 686.273891210556 seconds.
Beginning baseline calculation.
Baselines calculated in 32.12225961685181 seconds.
**********************************************************************************************
Processing subject 31 of 66
Save directory does not already exist.  Creating: K:\SV4\CW_17-11-04\L2-561nm-ROImonitoring_20171104_125048.corrected\extracted\rois_5_25_25
Beginning supervoxel extraction.
Searching for image files...
Found 6392 images.
Extracting: 2826 super voxels from 6392 images.
Extracted 2826 ROIS in 683.4338686466217 seconds.
Beginning baseline calculation.
Baselines calculated in 32.3450973033905 seconds.
**********************************************************************************************
Processing subject 32 of 66
Save directory does not already exist.  Creating: K:\SV4\CW_17-11-04\L3-561nm-ROImonitoring_20171104_144129.corrected\extracted\rois_5_25_25
Beginning su